# 🎬 Movie Buff Assistant with aisuite + MCP Tools

Build your own movie recommendation assistant that:
1. 🔍 Researches movies, actors, and directors
2. 🧠 Remembers your preferences and watch history
3. 💡 Gives personalized recommendations

## How This Works

When you pass MCP tools to aisuite:
1. **aisuite handles the glue work** - converts MCP tool specs to the format your LLM needs (OpenAI, Anthropic, etc.)
2. **Automatic execution** - when the LLM requests a tool, aisuite calls the MCP server and returns results
3. **You just write natural prompts** - no need to worry about tool schemas or execution logic!

This is the power of aisuite + MCP: unified tool calling across any LLM provider.

**What you need:**
- OpenAI API key (add to `.env` file)
- Python with `uv` installed (for fetch MCP server)
- Node.js/npx installed (for memory MCP server)

**Installation:**
```bash
pip install aisuite python-dotenv
pip install 'aisuite[mcp]'  # Includes MCP client + nest_asyncio for Jupyter support
pip install uv  # For fetch MCP server
```

In [1]:
import os
import sys
from pathlib import Path

# Add parent directory to Path - to pick up aisuite for development
# Skip this step if you're running from an installed package
repo_root = Path().absolute().parent.parent
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

from dotenv import load_dotenv
from aisuite import Client
from aisuite.mcp import MCPClient  # Needed to connect to MCP servers.

load_dotenv()

# Verify API key
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("Add OPENAI_API_KEY to .env file!")

print("✓ Ready to discover movies!")

✓ Ready to discover movies!


## Step 1: Set Up Your Movie Assistant

Provide tools from 2 different MCP servers to the LLM.
- **Fetch**: Get movie info from the web (IMDb, Wikipedia, reviews)
- **Memory**: Remember what movies you like and dislike

In [2]:
# Start fetch server (for getting movie data from the web)
fetch_mcp = MCPClient(
    command="uvx",
    args=["mcp-server-fetch"],
    name="fetch"
)

# Set up memory file for your movie preferences
memory_file = os.path.join(os.getcwd(), "movie_memory.jsonl")
# Start memory server (for remembering your preferences)
memory_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-memory"],
    env={"MEMORY_FILE_PATH": memory_file},
    name="memory"
)

print("Fetch server ready - can research movies from the web")
print("Memory server ready - will remember your preferences")
print(f"📁 Memories stored in: {memory_file}")

Fetch server ready - can research movies from the web
Memory server ready - will remember your preferences
📁 Memories stored in: /Users/rohit/fleet/leclerc/aisuite-prs/aisuite-main/aisuite/examples/agents/movie_memory.jsonl


## Step 2: Research a Movie & Store Your Opinion

Let's ask the assistant to research a movie and remember whether you liked it!

In [3]:
client = Client()

# Combine tools from both servers
all_tools = fetch_mcp.get_callable_tools() + memory_mcp.get_callable_tools()

response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{
        "role": "user",
        "content": """Research the movie 'Inception' from https://www.imdb.com/title/tt1375666/ or Wikipedia.
        
        Then:
        1. Store 'Inception' as a movie entity I loved
        2. Store that I like: complex plots, sci-fi, Christopher Nolan movies
        3. Add observations about why it's great (mind-bending, great visuals, etc.)
        4. Tell me 2-3 interesting facts you found
        
        Be enthusiastic and conversational!"""
    }],
    tools=all_tools,
    max_turns=10
)

print("="*60)
print("🎬 MOVIE RESEARCH")
print("="*60)
print(response.choices[0].message.content)

🎬 MOVIE RESEARCH
🎬 I'm thrilled to share some fascinating facts about *Inception*, a cinematic gem!

1. **A Global Shooting Journey**: Did you know that *Inception* was filmed in six different countries? The production kicked off in Tokyo and wrapped up in Canada, making it a truly international affair! 🌏

2. **A Decade in the Making**: Director Christopher Nolan initially penned an 80-page treatment for *Inception* after completing *Insomnia* back in 2002. But he decided to hone his craft with other projects, like *Batman Begins* and *The Dark Knight*, before finally bringing his dream-stealing concept to life. Talk about dedication! 📜✍️

3. **Mind-Bending Visuals**: The film is renowned for its stunning visual effects, especially the iconic scenes where the streets of Paris fold up like a mind-bending puzzle. It's a visual masterpiece that keeps you in awe! 🎥✨

I've enthusiastically recorded that *Inception* is a movie I love for its complex plot and visual brilliance. It aligns perf

## Step 3: Get Personalized Recommendations

Now ask for recommendations based on what it remembers about your tastes!

In [4]:
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{
        "role": "user",
        "content": """Tell me what you know about my movie preferences from memory, and suggest something new:
        
        1. Remind me what movies I liked
        2. Suggest 3 movies I'd probably enjoy
        3. Explain why each recommendation fits my taste
        
        Be enthusiastic like a friend recommending movies!"""
    }],
    tools=memory_mcp.get_callable_tools(),
    max_turns=10
)

print("="*60)
print("💡 PERSONALIZED RECOMMENDATIONS")
print("="*60)
print(response.choices[0].message.content)

💡 PERSONALIZED RECOMMENDATIONS
🎬 Hey there, movie buff! Here's a quick refresher on the movies you loved and your go-to interests:

**Movies You Loved:**
1. **Inception**: A mind-bending thrill ride directed by the genius Christopher Nolan. It's got all the layers, twists, and turns you love!
2. **Arrival**: This one’s a top recommendation for you because of its cerebral, first-contact story, wrapped in sci-fi brilliance!

**Your Movie Interests:**
1. **Complex Plots**: You love narratives that make you think and keep you on the edge of your seat.
2. **Sci-Fi**: Exploring futuristic themes and imaginative worlds is right up your alley.
3. **Christopher Nolan Movies**: His creative storytelling and direction style never fail to capture your attention.

🎥 **Movie Recommendations You'll Probably Enjoy:**

1. **Blade Runner 2049**: A breathtaking continuation of a sci-fi classic, directed by Denis Villeneuve. The movie's stunning visuals, intricate plot, and philosophical questions about h

## Step 4: Research a Director's Filmography

Let's explore a Director's work and see if the LLM can recommend something based on the limited preferences I saved earlier:

In [6]:
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{
        "role": "user",
        "content": """Research Denis Villeneuve's filmography from IMDb or Wikipedia.
        
        1. List his major films
        2. Based on my interests and moveis I liked earlier (check memory!), which of his films would I love?
        3. Store the top recommendation in memory
        
        Make it exciting - I love discovering new directors!"""
    }],
    tools=all_tools,
    max_turns=10
)

print("="*60)
print("🎥 DIRECTOR DEEP DIVE")
print("="*60)
print(response.choices[0].message.content)

🎥 DIRECTOR DEEP DIVE
### Denis Villeneuve's Major Films

Denis Villeneuve, a Canadian director, is renowned for his cerebral thrillers and large-scale science fiction films. Here is a list of his major films:

1. **Incendies (2010)** - A powerful drama exploring family secrets and the horrors of war.
2. **Prisoners (2013)** - A gripping thriller about a father's desperate search for his missing daughter.
3. **Enemy (2013)** - A psychological thriller exploring themes of identity and duality.
4. **Sicario (2015)** - A tense thriller about the drug war on the U.S.-Mexico border.
5. **Arrival (2016)** - A cerebral first-contact story about language, time, and choice.
6. **Blade Runner 2049 (2017)** - A visually stunning sequel to the classic sci-fi film, delving into the nature of humanity.
7. **Dune (2021)** - An epic adaptation of the renowned science fiction novel.
8. **Dune: Part Two (2024)** - The continuation of the epic sci-fi saga.

### Personalized Film Recommendation

Based on y

## Cleanup

In [ ]:
fetch_mcp.close()
memory_mcp.close()
print("✓ Servers closed - your movie preferences are saved!")

## Recap of what we did above

Behind the scenes, aisuite few things:
- ✅ **Converted MCP tool schemas** to OpenAI-compatible format
- ✅ **Handled tool execution** when the LLM requested them
- ✅ **Managed async operations** for MCP server communication

MCP Tools
**Web search** - LLM could do web search due to Fetch being passed as a tool.
**Preserved your data in knowledge graph** - LLM could store your preferences and look it using server-memory being passed as a tool.

You, as an user, just wrote natural prompts - aisuite handled all the tool calling complexity - LLM returned personalized recommendations.

You built a **personalized movie assistant** that:
- ✅ Researches movies from the web
- ✅ Learns your preferences over time
- ✅ Gives smart recommendations
- ✅ Remembers everything across sessions
- ✅ Builds a knowledge graph of your tastes

**All with minimal code!**

## Try These Next

Now that you've got the basics, explore more capabilities:

### 🎭 Refine Your Taste Profile
```python
# Tell it what you DON'T like to improve recommendations
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{"role": "user", "content": """Store in memory that I didn't enjoy:
        - 'Transformers' movies (too much action, not enough plot)
        - 'Scary Movie' series (slapstick comedy isn't my thing)
        
        Then suggest 3 movies I WOULD like based on my updated profile."""}],
    tools=memory_mcp.get_callable_tools(),
    max_turns=5
)
```

### 📊 Query Your Complete Watch History
```python
# See everything the assistant remembers
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{"role": "user", "content": """Search your memory and tell me:
        1. What movies do I love?
        2. What movies do I dislike?
        3. What are my key preferences (genres, themes, directors)?
        4. Based on all this, what's ONE perfect movie recommendation?"""}],
    tools=memory_mcp.get_callable_tools(),
    max_turns=5
)
```

### 🔍 Explore Your Memory File
```python
# See the raw knowledge graph
import json
with open(memory_file, 'r') as f:
    for line in f.readlines()[:10]:
        entry = json.loads(line)
        print(f"{entry.get('type')}: {entry.get('name')}")
```

### 🌟 More Fun Queries
- "Find me a thriller like 'Gone Girl'"
- "What are the best movies of 2024?"
- "Research Greta Gerwig's films and recommend one"
- "I'm in the mood for something uplifting - what should I watch?"
- "Based on my taste, should I watch [specific movie]?"
- "Find me a hidden gem from the 90s I might have missed"

## The Pattern

Setup MCP tools, and call chat.completions.create() with max_turns.

```python
# 1. Set up fetch + memory
fetch_mcp = MCPClient(command="uvx", args=["mcp-server-fetch"])
memory_mcp = MCPClient(
    command="npx",
    args=["-y", "@modelcontextprotocol/server-memory"],
    env={"MEMORY_FILE_PATH": "movie_memory.jsonl"}
)

# 2. Combine tools
tools = fetch_mcp.get_callable_tools() + memory_mcp.get_callable_tools()

# 3. Chat naturally!
response = client.chat.completions.create(
    model="openai:gpt-4o",
    messages=[{"role": "user", "content": "Research X and remember Y"}],
    tools=tools,
    max_turns=5
)
```

## Other Ideas

Use this same pattern for:
- 🍳 **Recipe assistant** that learns your dietary preferences (see `recipe_chef_assistant.ipynb`)
- ✈️ **Travel planner** that remembers your bucket list
- 📚 **Book recommender** that tracks your reading history
- 🎮 **Game advisor** that knows your favorite genres
- 🎵 **Music discovery** that learns your taste

## Resources

- **Fetch Server**: https://github.com/modelcontextprotocol/servers/tree/main/src/fetch
- **Memory Server**: https://github.com/modelcontextprotocol/servers/tree/main/src/memory
- **aisuite Documentation**: https://github.com/andrewyng/aisuite
- **More MCP Servers**: https://github.com/modelcontextprotocol/servers

**Your movie preferences persist across sessions** - restart this notebook anytime and your assistant will remember everything! 🎬✨
